In [1]:
import cv2
import os
import pandas as pd
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

In [2]:
confidence_threshold = 91  # Umbral de confianza en porcentaje

In [3]:
# Cargar el modelo entrenado
model_path = '../models/reconocimiento_facial_15_personasV2.keras'
if not os.path.exists(model_path):
    raise FileNotFoundError(f"El modelo no se encontró en la ruta: {model_path}")
model = load_model(model_path)

# Clase de nombres de las 15 personas
class_names = ['Anderson', 'Anibal', 'Carlos', 'Cesar', 'David', 'Diego', 'Fabricio',
               'Ivan', 'Johan', 'Jose', 'Justmary', 'Kevin', 'Luisa', 'Teresa', 'Wong']  # Cambiar según corresponda

# Inicializar la cámara web
cap = cv2.VideoCapture(1)

# Cargar Haar Cascade para la detección de rostros
cascade_path = '../models/haarcascade_frontalface_alt.xml'
if not os.path.exists(cascade_path):
    raise FileNotFoundError(f"El clasificador Haar Cascade no se encontró en la ruta: {cascade_path}")
face_cascade = cv2.CascadeClassifier(cascade_path)

# Crear el archivo .csv si no existe
csv_file = '../results/asistencia.csv'
if not os.path.exists(csv_file):
    # Crear un DataFrame vacío con columnas "Nombre" y "Fecha"
    df = pd.DataFrame(columns=["Nombre", "Fecha"])
    df.to_csv(csv_file, index=False)

# Definir la función para registrar la asistencia
def registrar_asistencia(nombre):
    df = pd.read_csv(csv_file)
    fecha_actual = pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')
    nuevo_registro = pd.DataFrame({"Nombre": [nombre], "Fecha": [fecha_actual]})
    df = pd.concat([df, nuevo_registro], ignore_index=True)
    df.to_csv(csv_file, index=False)

In [ ]:
# Código para realizar predicciones en tiempo real
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Convertir el frame a escala de grises
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detectar rostros en el frame
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    for (x, y, w, h) in faces:
        # Extraer el rostro detectado
        face = frame[y:y+h, x:x+w]
        
        # Preprocesar el rostro para la predicción
        face_array = cv2.resize(face, (224, 224))
        face_array = np.expand_dims(face_array, axis=0)
        face_array = face_array / 255.0  # Normalizar

        # Realizar predicción
        try:
            predictions = model.predict(face_array)
        except Exception as e:
            print(f"Error al realizar la predicción: {e}")
            continue

        # Obtener el índice de la clase con la mayor probabilidad
        predicted_class = np.argmax(predictions)

        # Verificar que el índice de la clase esté dentro del rango
        if predicted_class >= len(class_names):
            print(f"Índice de clase fuera de rango: {predicted_class}")
            continue

        # Obtener el nombre de la persona y la precisión de la predicción
        person_name = class_names[predicted_class]
        confidence = np.max(predictions) * 100  # Convertir a porcentaje

        # Mostrar el nombre y la precisión sobre el rostro en el video
        label = f"{person_name}: {confidence:.2f}%"
        cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

        # Registrar la asistencia de la persona si la confianza es mayor al umbral
        if confidence > confidence_threshold:
            try:
                registrar_asistencia(person_name)
            except Exception as e:
                print(f"Error al registrar la asistencia: {e}")

        # Dibujar un rectángulo alrededor del rostro detectado
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

    # Mostrar el video en tiempo real
    cv2.imshow('Reconocimiento Facial en Tiempo Real', frame)

    # Salir del bucle si se presiona la tecla 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Liberar la cámara y cerrar las ventanas
cap.release()
cv2.destroyAllWindows()

# Procesamiento del archivo CSV para quedarse con la primera y última hora registrada de cada persona
df = pd.read_csv(csv_file)

# Convertir la columna 'Fecha' a tipo datetime
df['Fecha'] = pd.to_datetime(df['Fecha'])

# Ordenar el DataFrame por 'Nombre' y 'Fecha'
df = df.sort_values(by=['Nombre', 'Fecha'])

# Crear un nuevo DataFrame para almacenar los registros procesados
df_processed = pd.DataFrame(columns=["Nombre", "Fecha"])

# Procesar el DataFrame para quedarse solo con la primera y última hora registrada de cada persona
for nombre in df['Nombre'].unique():
    df_persona = df[df['Nombre'] == nombre]
    primera_hora = df_persona.iloc[0]
    ultima_hora = df_persona.iloc[-1]
    df_processed = pd.concat([df_processed, pd.DataFrame([primera_hora])], ignore_index=True)
    if not primera_hora.equals(ultima_hora):
        df_processed = pd.concat([df_processed, pd.DataFrame([ultima_hora])], ignore_index=True)

# Guardar el DataFrame procesado en un nuevo archivo CSV
df_processed.to_csv('../results/asistencia.csv', index=False)

print("\n\nEl archivo 'asistencia.csv' ha sido procesado con éxito.")